In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# 格納フォルダ
fileDir = r'C:\Users\u647088\Desktop'

# 投薬データファイル
inFile = 'yakuzaiData.tsv' 


In [3]:
def extractLine(df):
    def __extractLine(tDf, kai, syu):
        # 薬剤ごとの期間の算出
        sDf = tDf[
            (((kai <= tDf['kaibi']) & (tDf['kaibi'] <= syu)))].groupby(
                'yakuzai', as_index=False).agg(
                    {'kaibi': 'min', 'syubi': 'max'}).copy()
        # 開始日による暫定治療ラインの分割
        periodSet = set(sDf['kaibi'].tolist())
        tSet = set()
        for tDate in periodSet:
            # 開始日が暫定治療ライン内に存在したら分割
            if kai < tDate <= syu and kai != tDate:
                tSet = tSet | __extractLine(
                    tDf, kai, tDate + timedelta(days=-1))
                tSet = tSet | __extractLine(tDf, tDate, syu)
        if len(tSet) == 0:
            tSet.add((kai, syu))
        return tSet

    def __extractLineBCG(tDf):
        if len(tDf) == 0:
            return tDf
        # 開始日による暫定治療ラインの分割
        periodList = tDf['kaibi'].tolist()
        periodList.sort()
        kaiList = []
        shuList = []
        yakuList = []
        kaiDate = None 
        shuDate = None 
        for tDate in periodList:
            # 初回は必ず設定
            if kaiDate is None:
                kaiDate = tDate
                shuDate = tDate + timedelta(days=20)
            # 開始日が21日後以上だったら分割
            elif shuDate < tDate:
                kaiList.append(kaiDate)
                shuList.append(shuDate)
                yakuList.append('乾燥ＢＣＧ膀胱内用_' + str(len(kaiList)))
                kaiDate = tDate
                shuDate = tDate + timedelta(days=20)
            # 開始日が21日後未満だったら終了日を更新
            else:
                shuDate = tDate + timedelta(days=20)
                
        kaiList.append(kaiDate)
        shuList.append(shuDate)
        yakuList.append('乾燥ＢＣＧ膀胱内用_' + str(len(kaiList)))
        return pd.DataFrame({'yakuzai' : yakuList, 'kaibi' :kaiList, 'syubi' :shuList})

    # 全体の期間の算出
    tDf = df.copy()
    tDf['kaibi'] = pd.to_datetime(tDf['kaibi'], format='%Y%m%d')
    tDf['syubi'] = pd.to_datetime(tDf['syubi'], format='%Y%m%d')

    # BCGの暫定ライン作成（必要な場合、下記2行も実行する）
    sDf2 = __extractLineBCG(tDf[tDf['yakuzai'] == '乾燥ＢＣＧ膀胱内用'])
    tDf = pd.concat([tDf[tDf['yakuzai'] != '乾燥ＢＣＧ膀胱内用'].copy(), sDf2])
    
    kaibi = tDf['kaibi'].min()
    syubi = tDf['syubi'].max()

    datelist = __extractLine(tDf, kaibi, syubi)

    # 日付の文字列化、開始日の昇順でソート
    datelist = [(kai.strftime('%Y%m%d'),
                 syu.strftime('%Y%m%d')) for kai, syu in datelist]
    datelist = sorted(datelist, key=lambda x: x[0])

    # 治療ラインごとの集計
    rDf = pd.DataFrame()
    for i, l in enumerate(datelist):
        # 治療ラインの期間内に投薬開始日となっている薬剤で集計
        tDf = df[(((l[0] <= df['kaibi']) & (df['kaibi'] <= l[1])))].copy()
        tDf = tDf.groupby('yakuzai', as_index=False).agg(
            {'kaibi': 'min', 'syubi': 'max'})
        tDf['治療ライン'] = i + 1
        tDf['同一治療ライン上の薬剤数'] = len(tDf)
        # 薬剤の投薬終了日が治療ラインの投薬終了日より後になっていた場合、
        # 当治療ラインの終了日にする
        tDf['syubi'] = tDf['syubi'].apply(
            lambda x: l[1] if l[1] < x else x)

        # 最初の治療ライン以外の場合
        if i != 0:
            # 前の治療ラインに同一薬剤が存在したら「確認対象」とする
            for yakuzai in tDf['yakuzai']:
                cDf = tDf[tDf['yakuzai'] == yakuzai].copy()
                if len(rDf[rDf['yakuzai'] == yakuzai]) != 0:
                    cDf['確認対象フラグ'] = '確認対象'
                else:
                    cDf['確認対象フラグ'] = '確認対象外'
                rDf = pd.concat([rDf, cDf])
        else:
            tDf['確認対象フラグ'] = '確認対象外'
            rDf = pd.concat([rDf, tDf])

    return rDf

In [5]:
"""
**メイン処理**

# テストデータの読込
# 対象データ抽出（CSV読み込み）
# 患者ごとの治療ライン導出処理実行
# 処理結果ファイルの出力
"""

# テストデータの読込
dataFile = os.path.join(fileDir, inFile)
df = pd.read_table(dataFile, dtype=str, encoding='utf8')
df = df.fillna('')

# 患者ごとの処理実行
rDf = pd.DataFrame()
for kanja in df[~df['kanja'].duplicated()]['kanja']:
    tDf = extractLine(df[df['kanja'] == kanja].copy().drop(
        ['kanja', 'kanja_id', 'facility'], axis=1))
    dKanja = kanja.split('__')
    tDf['facility'] = dKanja[0]
    tDf['kanja_id'] = dKanja[1]
    rDf = pd.concat([rDf, tDf])

# 結果出力
rDf.sort_values(
    ['facility', 'kanja_id', '治療ライン', 'kaibi', 'yakuzai'],
    inplace=True)
# 処理結果ファイルの出力
file = os.path.join(fileDir, 'yakuzaiLine_' + "{0:%Y%m%d%H%M}".format(datetime.now()) + '.tsv')
rDf.to_csv(file, encoding='cp932', sep='\t', index=False, header=True,
           columns=['facility', 'kanja_id', 'yakuzai', 'kaibi',
                    'syubi', '治療ライン', '確認対象フラグ',
                    '同一治療ライン上の薬剤数'])


In [88]:
"""
**治療ラインの作成処理**

# 治療ラインNo,治療ライン名の作成
"""

def createLine(df):
    df.set_index('kaibi', inplace=True)
    rowCountSeries = df.index.value_counts()

    rDf = pd.DataFrame()
    linename = ''
    for tmpKaibi in set(df.index.values):
        # 同一開始日に1行しか存在しないレコードの場合
        if rowCountSeries[tmpKaibi] == 1:
            tDf = df.loc[tmpKaibi].to_frame()
            tDf = tDf.T.reset_index(drop=True)
        else:
            tDf = df.loc[tmpKaibi]
            tDf.reset_index(drop=True, inplace=True)
        tmpLinename = '_'.join(sorted(set(tDf['yakuzai'])))
        print(tmpLinename)
    return df
    

In [89]:
# テストデータの読込
dataFile = os.path.join(fileDir, inFile)
df = pd.read_table(dataFile, dtype=str, encoding='utf8')
df = df.fillna('')
df['kaibi'] = pd.to_datetime(df['kaibi'], format='%Y%m%d')
df['syubi'] = pd.to_datetime(df['syubi'], format='%Y%m%d')

# 患者ごとの処理実行
rDf = pd.DataFrame()
df.set_index('kanja_id', inplace=True)
rowCountSeries = df.index.value_counts()
for tmpKanjaId in set(df.index.values):
    # 1行しか存在しないレコードの場合
    if rowCountSeries[tmpKanjaId] == 1:
        tDf = df.loc[tmpKanjaId].to_frame()
        tDf = tDf.T.reset_index(drop=True)
    else:
        tDf = df.loc[tmpKanjaId]
        tDf.reset_index(drop=True, inplace=True)
    lineDf = createLine(tDf)
    print(lineDf)


乾燥ＢＣＧ膀胱内用
膀胱悪性腫瘍手術（経尿道的手術）
ドキソルビシン塩酸塩_膀胱悪性腫瘍手術（経尿道的手術）
             facility           yakuzai      syubi
kaibi                                             
2018-06-19  120000001         乾燥ＢＣＧ膀胱内用 2018-06-19
2020-02-14  120000001  膀胱悪性腫瘍手術（経尿道的手術） 2020-02-14
2020-10-20  120000001  膀胱悪性腫瘍手術（経尿道的手術） 2020-10-20
2020-10-20  120000001        ドキソルビシン塩酸塩 2020-10-20
乾燥ＢＣＧ膀胱内用
             facility    yakuzai      syubi
kaibi                                      
2017-11-13  120000001  乾燥ＢＣＧ膀胱内用 2017-11-13
乾燥ＢＣＧ膀胱内用
             facility    yakuzai      syubi
kaibi                                      
2020-01-20  120000001  乾燥ＢＣＧ膀胱内用 2020-01-20
乾燥ＢＣＧ膀胱内用
             facility    yakuzai      syubi
kaibi                                      
2019-12-20  120000001  乾燥ＢＣＧ膀胱内用 2019-12-20


In [75]:
s = ['b', 'a', 'c', 'a', 'c']
'_'.join(sorted(set(s)))

'a_b_c'